### **Mustache template y search**

Este notebook muestra el desarrollo del template con sus funcionalidades de busqueda y filtrado. Además, se presenta una sección para probar la función search, que posteriormente se incluirá en la app.py de flask

**Librerias**

In [2]:
from flask import Flask, request, jsonify
from elasticsearch import Elasticsearch
import json

**Configuración de Elasticsearch y Flask**

In [2]:
# Establecer conexión ES
es = Elasticsearch("http://localhost:9200")
# Crear la aplicación Flask
app = Flask(__name__)


### **Endpoint 1: Búsqueda Avanzada con Template Mustache**

Se incluye a 2 tipos de variables para ser consultadas en el template, ademas, como parte de la solución del punto 3.4, el template incluye una priorizacion en los resultados a los productos con mejores rating de los clientes y a productos con más de 100 ratings.

**El template que se creará permitirá hacer búsquedas por nombre de producto (name) usando coincidencia difusa y aplicar filtros exactos por categoría principal (main_category). Se almacena en Elasticsearch con el nombre "amazon_search_template" para ser reutilizado en futuras consultas.**

In [1]:
# template Mustache 
template_body = {
  "script": {
    "lang": "mustache",
    "source": """
    {
      "query": {
        "bool": {
          "must": [
            { "match": { "name": "{{search_box}}" }},
            { "term": { "main_category": "{{main_category}}" }}
          ],
          "should": [
            { "range": { "ratings": { "gte": 4.0, "boost": 2.0 } } },
            { "range": { "no_of_ratings": { "gte": 100, "boost": 1.5 } } }
          ]
        }
      }
    }
    """
  }
}

# Guardar el template en Elasticsearch
template_name = "amazon_search_template"
es.put_script(id=template_name, body=template_body)
print(f" Template {template_name} creado exitosamente.")

NameError: name 'es' is not defined

In [ ]:
def search():
    search_box = request.args.get('search_box', '')
    main_category = request.args.get('main_category', '')

    # Solicitar búsqueda con template
    response = es.search_template(
        index="amazon_products",
        body={
            "id": "amazon_search_template",
            "params": {
                "search_box": search_box,
                "main_category": main_category
            }
        }
    )

    # Procesar resultados
    results = [
        {
            "name": hit["_source"]["name"],
            "ratings": hit["_source"].get("ratings", "N/A"),
            "no_of_ratings": hit["_source"].get("no_of_ratings", "N/A"),
            "discount_price_dolares": hit["_source"].get("discount_price_dolares", "N/A"),
            "actual_price_dolares": hit["_source"].get("actual_price_dolares", "N/A")
        }
        for hit in response["hits"]["hits"]
    ]

    return jsonify(results)

**Probando la función search**

In [ ]:
resultados = search(es, template_name="amazon_search_template", search_box="Adidas", main_category="sports & fitness")
# Mostrar los resultados
for resultado in resultados[:5]:  # Mostrar solo los 5 primeros resultados
    producto = resultado["_source"]
    print(f" Producto: {producto['name']}")
    print(f" Categoría: {producto['main_category']}")
    print(f" Precio con Descuento: {producto['discount_price_dolares']}")
    print(f" Calificación: {producto['ratings']}")
    print(f" Link: {producto['link']}\n")